In [1]:
import os
import cv2
import numpy as np
import pickle
from scipy.io import loadmat
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn.cluster import KMeans
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.neighbors import KDTree

In [2]:
data_path        = './2015年全国研究生数学建模竞赛B题/数据'
exercise1_fname  = '第1题数据/1.mat'
exercise2a_fname = '第2题数据/2a.mat'
exercise2b_fname = '第2题数据/2b.mat'
exercise2c_fname = '第2题数据/2c.mat'
exercise2d_fname = '第2题数据/2d.mat'
exercise3a_fname = '第3题数据/3a.mat'
exercise3b_fname = '第3题数据/3b.mat'
exercise3c_fname = '第3题数据/3c.mat'
exercise4a_fname = '第4题数据/4a.mat'
exercise4b_fname = '第4题数据/4b.mat'

def load_mat(filename):
    data = loadmat(filename)['data'].T
    return data
def show2dfig(X, labels=None):
    if X.shape[1] != 2: print('dimension error!'); return
    plt.figure()
    plt.scatter(X[:, 0], X[:, 1], c=labels)
    plt.show()
def show3dfig(X, labels=None):
    if X.shape[1] != 3: print('dimension error!'); return
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=labels)
    plt.show()
angle = lambda x, y: np.arccos(x.dot(y) / (np.linalg.norm(x)*np.linalg.norm(y))) * 180 / np.pi

In [3]:
X = load_mat(os.path.join(data_path, exercise4a_fname))
# vecs = np.load("./data/vec.npy")
vecs = loadmat('./2015年全国研究生数学建模竞赛B题/Xs10.mat')['X1'].T
pca = PCA(n_components=3)
X = pca.fit_transform(X)

In [4]:
n_samples = X.shape[0]
n_features = X.shape[1]
y = np.zeros(shape=(n_samples,))
k = 10; d=0.15; thresh=5
y = np.zeros(n_samples)
Xrest = np.c_[X, np.zeros(n_samples), np.arange(n_samples)]
nowMax = 1
while Xrest[Xrest[:, 3]==0].shape[0] > 0:
#     print_log = "剩余{:.2%}未标记".format(Xrest[Xrest[:, 3]==0].shape[0] / n_samples)
#     print(print_log)
    nowId = 0
    x0 = Xrest[nowId, :3]
    v0 = vecs[Xrest[nowId, 4].astype('int')]
    while True:
        # 搜索d近邻内的点
        kdtree = KDTree(Xrest[:, :3], metric='euclidean')
        k_ = Xrest.shape[0] if Xrest.shape[0] < k else k
        dist, idx = kdtree.query(x0.reshape((1, -1)), k=k_)
        dist = dist.reshape(-1); idx = idx.reshape(-1)
        idx = idx[dist<d]
        # 筛选角度小于阈值的点
        for i in range(idx.shape[0]):
            idx_ = idx[i]
            v1 = vecs[int(Xrest[idx_, 4])]
            ang = angle(v0, v1)
            if (ang < thresh) or (ang > 180 - thresh):
                y[int(Xrest[idx_, -1])] = nowMax
        # 移除当前点
        Xrest = np.delete(Xrest, nowId, axis=0)
        # 搜索同类别下一个点
        if Xrest[Xrest[:, 3]==nowMax].shape[0] > 0:
            for i in range(Xrest.shape[0]):
                if Xrest[i, 3] == nowMax:
                    nowId = i
                    x0 = Xrest[i, :3]
                    v0 = vecs[int(Xrest[nowId, 4])]
                    break
        else:
            nowMax += 1
            break

c:\myapplications\python3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in arccos


In [5]:
cnt = np.bincount(y.astype('int'))
mainmark = np.argsort(cnt)[::-1][:3]

In [6]:
s = np.sort(cnt)[::-1]
s[s>0][:10]

array([777,   4,   4,   4,   4,   4,   4,   4,   4,   3], dtype=int64)